In [0]:
%run "./Includes/dependencies"

In [0]:
#Limpieza de cache
spark.catalog.clearCache()
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

## 1. Carga de informacion

### Carga de toda la informacion

In [0]:
for bronze_path in bronze_paths:
    globals()[bronze_path["dataframe_name"]] = (
        spark.read.format("parquet")
        .option("recursiveFileLookup", "true")
        .load(bronze_path["path"])
    )
    print(f'{bronze_path["dataframe_name"]}\t{globals()[bronze_path["dataframe_name"]].count()}')

contacto_df	324511
donaciones_df	120145
grados_df	150507
comunidad_df	72584
eventos_df	382044
revista_df	29347
laboral_df	324511
relaciones_df	173194

In [0]:
# Agrega metadato origen de ubicacion a dataframes
contacto_df = contacto_df.withColumn('origen', F.input_file_name())
donaciones_df = donaciones_df.withColumn('origen', F.input_file_name())
grados_df = grados_df.withColumn('origen', F.input_file_name())
comunidad_df = comunidad_df.withColumn('origen', F.input_file_name())
eventos_df = eventos_df.withColumn('origen', F.input_file_name())
revista_df = revista_df.withColumn('origen', F.input_file_name())
laboral_df = laboral_df.withColumn('origen', F.input_file_name())
relaciones_df = relaciones_df.withColumn('origen', F.input_file_name())

In [0]:
#display(donaciones_df)

### Conteo de registros por origen

In [0]:
# Conteo de registros por origen.

schema = StructType([
    StructField('origen',StringType(),True),
    StructField('conteo_registros',LongType(),False)
])
conteos_df = spark.createDataFrame([], schema=schema)
for bronze_path in bronze_paths:
    temp_df = (
        globals()[bronze_path["dataframe_name"]]
         .select('origen')
         .groupBy('origen')
         .count()
         .select(F.col('origen'), F.col('count').alias('conteo_registros'))
         .orderBy(F.col('origen'), F.col('conteo_registros'))
        )
    conteos_df = conteos_df.union(temp_df)

display(conteos_df)


origen,conteo_registros
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,324511
dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet,120145
dbfs:/mnt/donaciones/Bronze/INFO_GRADOS.parquet,150507
dbfs:/mnt/donaciones/Bronze/INFO_INTERACCION_COMUNIDAD.parquet,72584
dbfs:/mnt/donaciones/Bronze/INFO_INTERACCION_EVENTOS.parquet,382044
dbfs:/mnt/donaciones/Bronze/INFO_INTERACCION_REVISTA.parquet,29347
dbfs:/mnt/donaciones/Bronze/INFO_LABORAL.parquet,324511
dbfs:/mnt/donaciones/Bronze/INFO_RELACIONES.parquet,173194


### Conteo de nulos por origen y columna

In [0]:
for bronze_path in bronze_paths:
    print(bronze_path["dataframe_name"])
    copy_df = globals()[bronze_path["dataframe_name"]]
    columns = copy_df.columns
    for column in columns:
        copy_df=copy_df.withColumn(column,
            F.when(F.col(column)== 'NULL',
                   None
                  )
            .otherwise(
                F.col(column)
            ))
        globals()[bronze_path["dataframe_name"]]= copy_df

contacto_df
donaciones_df
grados_df
comunidad_df
eventos_df
revista_df
laboral_df
relaciones_df

In [0]:

schema = StructType([
  StructField("source", StringType() ,True),
  StructField("col_name", StringType() ,True),
  StructField("count", IntegerType() ,True),
  StructField("count_null", IntegerType() ,True),
  StructField("precent_null", FloatType() ,True)
])

nullSdf = spark.createDataFrame([],schema)

for bronze_path in bronze_paths:
    print(bronze_path["dataframe_name"])
    copy_df = globals()[bronze_path["dataframe_name"]]
    source_list = copy_df.select('origen').distinct().collect()
    for source_item in source_list:
        tempSdf = null_percent(copy_df.where(col('origen') == source_item['origen']).drop('origen'))
        tempSdf = tempSdf.withColumn("source" , lit(source_item['origen']))  
        tempSdf = tempSdf.select("source" , "col_name" , "count" , "count_null" , "precent_null")
        nullSdf = nullSdf.union(tempSdf)


contacto_df
donaciones_df
grados_df
comunidad_df
eventos_df
revista_df
laboral_df
relaciones_df

In [0]:
display(nullSdf.orderBy(F.col('source'), F.col('precent_null').desc()))

source,col_name,count,count_null,precent_null
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,PUNTAJE_RFM,19899.0,304612.0,93.8680044744246
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,ESTRATO,115868.0,208643.0,64.29458477524645
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,ESTADO_CIVIL,137144.0,187367.0,57.738258487385636
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,GENERACION,166079.0,158432.0,48.821765672041934
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,EDAD,166223.0,158288.0,48.77739121324085
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,FECHA_NACIMIENTO,166223.0,158288.0,48.77739121324085
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,DIRECCION,206899.0,117612.0,36.242839225788956
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,DEPARTAMENTO,240724.0,83787.0,25.819463746991627
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,PAIS,240941.0,83570.0,25.7525939028261
dbfs:/mnt/donaciones/Bronze/INFO_CONTACTO.parquet,CIUDAD,241136.0,83375.0,25.69250348986629


## 2. Consolidación de información

In [0]:
#Se crea una tabla consolidada de donaciones
donaciones_df.createOrReplaceTempView("donaciones")

In [0]:
%sql
select * from donaciones

ID_CONTACTO,ID_DONACION,ID_OPPORTUNIDAD,ID_CUENTA_CONTABLE,CUENTA_CONTABLE,ESTADO_DE_PROMESA,FECHA_DE_PAGO,MONTO_DONADO,origen
0031I00000CM12nQAD,a091I00000ZnGIWQA3,0061I00000QLCegQAH,a0e1I00000DvMLtQAN,FONDO NECESITAMOS PENSAR,Closed Won,2021-08-20,250000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet
0031I00001D60dOQAR,a091I00000Y5Nz7QAF,0061I00000OhzGiQAJ,a0e1I00000DKTtpQAH,FONDO VAMOS PALANTE,Closed Won,2020-10-07,10000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet
0031I00000Aj1OWQAZ,a091I00000ZnGIMQA3,0061I00000QLCebQAH,a0e1I00000DvMLtQAN,FONDO NECESITAMOS PENSAR,Closed Won,2021-08-20,50000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet
0031I00000UvQHWQA3,a091I00000ZmCg1QAF,0061I00000QK7sBQAT,a0e1I000002PsGsQAK,FONDO PALANTE PACÍFICO,Closed Won,2021-06-15,20000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet
0031I00000AislAQAR,a091I00000Y5NxBQAV,0061I00000OhzGgQAJ,a0e1I00000DKTtpQAH,FONDO VAMOS PALANTE,Closed Won,2020-10-06,50000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet
0031I00000AifsHQAR,a091I00000W0diWQAR,0061I00000QK7UEQA1,a0e1I000002PsF4QAK,Quiero Estudiar Beca con Compromiso,Closed Won,2021-07-12,80000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet
0031I00000AixcYQAR,a098W00000ZnofbQAB,0068W00000QM2Z7QAL,a0e1I00000DvcrHQAR,FONDO VAMOS PALANTE 2021,Closed Won,2021-10-15,100000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet
0031I00000Aix9EQAR,a091I00000BgXcDQAV,0061I00000QJPNuQAP,a0e1I000002PsGsQAK,FONDO PALANTE PACÍFICO,Closed Won,2021-05-06,30000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet
0031I00000CMJ0pQAH,a091I00000Y5NtOQAV,0061I00000OiCmJQAV,a0e1I00000DKYXAQA5,FONDO VAMOS PALANTE FRANCISCO LEAL,Closed Won,2020-10-17,8800000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet
0031I00001FEHRFQA5,a098W00000ZnohUQAR,0068W00000QM2bmQAD,a0e1I00000DvcrHQAR,FONDO VAMOS PALANTE 2021,Closed Won,2021-10-21,500000,dbfs:/mnt/donaciones/Bronze/INFO_DONACIONES.parquet


In [0]:
%sql
create Or Replace Temp View donacionesconsolidado AS
SELECT ID_CONTACTO, SUM(MONTO_DONADO) MONTO_TOTAL, COUNT(DISTINCT ID_DONACION) DONACIONES, COUNT(DISTINCT ID_OPPORTUNIDAD) OPORTUNIDADES
FROM donaciones
GROUP BY ID_CONTACTO

In [0]:
%sql
SELECT *
FROM donacionesconsolidado

ID_CONTACTO,MONTO_TOTAL,DONACIONES,OPORTUNIDADES
0031I00000CM7dvQAD,300000.0,1,1
0031I00000AizqQQAR,2370000.0,8,73
0031I00000CMDXUQA5,2395000.0,6,100
0031I00000Aiv8gQAB,1.5629652E7,8,46
0031I00000AiyJ0QAJ,1020000.0,3,34
0031I00000CMG0tQAH,1665000.0,3,48
0031I00000CMIPyQAP,5503924.0,18,123
0031I00000AisI6QAJ,1620000.0,17,139
0031I00000CMJP5QAP,2800000.0,3,51
0031I00000AilMCQAZ,4.7E7,4,4


In [0]:
donacionesconsol_df= spark.sql('select * from donacionesconsolidado')

In [0]:
#Joins con tablas de contacto, laboral y consolidado de donaciones.
final_df = (
   contacto_df
    .join(laboral_df,'ID_CONTACTO','left')
    .drop('origen')
    .join(donacionesconsol_df,'ID_CONTACTO','left')
)
print(final_df.select('ID_CONTACTO').distinct().count())

324511

In [0]:
display(final_df)

ID_CONTACTO,ESTADO_CIVIL,FECHA_NACIMIENTO,EDAD,GENERACION,GENERO,PAIS,CIUDAD,DEPARTAMENTO,DIRECCION,ES_GRADUADO,ES_EXALUMNO,ES_ANTIGUO,ES_ESTUDIANTE,ES_MIEMBRO_CONCEJO,ES_PADRE_MADRE,ES_DONANTE,ES_RESPONSABLE_FINANCIERO,ES_ADMINISTRATIVO,ES_PROFESOR,ES_ANTIGUO_MIEMBRO_CONSEJO_SUPERIOR,ES_DONANTE_LOS_ANDES_FOUNDATION,ES_PROFESOR_ADMINISTRATIVO_RETIRADO,VOLUNTARIO_UNIANDES,ES_EGRESADO_CERTIFICADO,ESTRATO,PUNTAJE_RFM,CARGO,NIVEL_JERARQUICO,SECTOR_ECONOMICO_1,SECTOR_ECONOMICO_2,EMPLEADOR,NIT,Empleador__c,MONTO_TOTAL,DONACIONES,OPORTUNIDADES
0031I00000AibubQAB,S - SOLTERO,1986-03-01,36,Generación Y (Millennials),Masculino,COLOMBIA,BOGOTA D.C.,BOGOTA,CL 56 71 54,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,null,ASESOR FINANCIERO,Profesional,Actividades jurídicas y de contabilidad,"Actividades profesionales, científicas y técnicas",CARRIZOSA CONSULTORES SAS,9005276702,0011I00001PhB8jQAF,null,null,null
0031I00000AibweQAB,null,null,null,null,Femenino,COLOMBIA,BOGOTA D.C.,BOGOTA,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null
0031I00000AibwHQAR,null,null,null,null,Femenino,null,null,null,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null
0031I00000AibwkQAB,null,null,null,null,Masculino,null,null,null,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null
0031I00000AibwLQAR,S - SOLTERO,1998-09-30,24,Generación Z (Centennials),Femenino,COLOMBIA,CAJICA,CUNDINAMARCA,VRD 12 MCP CAJICA CA 304 CN PUERTA DEL SOL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,null,null,null,null,null,null,null,null,null,null,null
0031I00000AibwNQAR,null,null,null,null,Masculino,null,null,null,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null
0031I00000AibwsQAB,null,null,null,null,Femenino,null,null,null,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null
0031I00000Aibx1QAB,S - SOLTERO,1998-01-30,24,Generación Z (Centennials),Masculino,COLOMBIA,BOGOTA D.C.,BOGOTA,CL 147 7 B 37 AP 1303,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,6,111,null,null,null,null,null,null,null,4000.0,1,1
0031I00000Aibx4QAB,null,null,null,null,Femenino,null,null,null,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null
0031I00000Aibx6QAB,null,null,null,null,Masculino,null,null,null,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null


### Validación valores unicos por columnas

In [0]:
#Obtiene total registros
total_registros = final_df.count()
    
#Mostrar información por cada columna

for column in final_df:
    displayHTML(f'<h3>{column}</h3>')
    display(final_df.select(column).groupBy(column).count().withColumn('porcentaje', F.round((F.col('count') / total_registros)*100.0, 2)))


Column<'ID_CONTACTO'>

ID_CONTACTO,count,porcentaje
0031I00000CMFbUQAX,1,0.0
0031I00000UvQI8QAN,1,0.0
0031I00000H2L4FQAV,1,0.0
0031I00000tX3OaQAK,1,0.0
0031I00000tX3tWQAS,1,0.0
0031I00000rJQbWQAW,1,0.0
0031I00000rJQC8QAO,1,0.0
0031I00000rJQCnQAO,1,0.0
0031I00000jTAAfQAO,1,0.0
0031I00000UuX3fQAF,1,0.0


Column<'ESTADO_CIVIL'>

ESTADO_CIVIL,count,porcentaje
null,187367,57.74
C - CASADO,38518,11.87
U - UNION LIBRE,1841,0.57
S - SOLTERO,95067,29.3
D - DIVORCIADO,766,0.24
P - SEPARADO,779,0.24
V - VIUDO,173,0.05


Column<'FECHA_NACIMIENTO'>

...

MONTO_TOTAL,count,porcentaje
300000.0,320,0.1
330000.0,39,0.01
4800.0,1,0.0
495000.0,2,0.0
7460000.0,1,0.0
5010000.0,2,0.0
1185000.0,2,0.0
4.138E7,1,0.0
1.075E7,1,0.0
7.45E7,1,0.0


Column<'DONACIONES'>

DONACIONES,count,porcentaje
26,8,0.0
29,9,0.0
19,12,0.0
54,1,0.0
0,362,0.11
22,12,0.0
7,260,0.08
34,5,0.0
32,3,0.0
31,4,0.0


Column<'OPORTUNIDADES'>

OPORTUNIDADES,count,porcentaje
29,23,0.01
26,26,0.01
65,15,0.0
191,1,0.0
54,20,0.01
19,28,0.01
113,4,0.0
155,1,0.0
112,2,0.0
167,1,0.0


### Conversion tipo de datos columnas

In [0]:
#Definicion de columnas enteras y fechas.
enteros = ['MONTO_TOTAL','ES_ADMINISTRATIVO','ES_ANTIGUO','ES_ESTUDIANTE','ES_EXALUMNO','ES_GRADUADO','ES_MIEMBRO_CONCEJO','ES_PADRE_MADRE','ES_PROFESOR','ES_RESPONSABLE_FINANCIERO','EDAD'] 
fechas = [
    {
        'columna': 'FECHA_NACIMIENTO',
        'formato': 'yyyy-MM-dd'
    }
]


In [0]:
#Procesamiento de columnas de numeros enteros
for columna_entero in enteros:
    final_df = final_df.withColumn(columna_entero, F.col(columna_entero).cast('int'))

In [0]:
final_df.createOrReplaceTempView('TBLFIN')

In [0]:
%sql
select SUM (MONTO_TOTAL)FROM TBLFIN 
-- ,SUM('ES_ADMINISTRATIVO'),SUM('ES_ANTIGUO'),SUM('ES_ESTUDIANTE'),SUM('ES_EXALUMNO'),SUM('ES_GRADUADO'),SUM('ES_MIEMBRO_CONCEJO'),SUM('ES_PADRE_MADRE'),SUM('ES_PROFESOR'),SUM('ES_RESPONSABLE_FINANCIERO'),SUM('ASISTIO'),SUM('INSCRITO')  FROM TBLFIN

sum(MONTO_TOTAL)
22137091978


In [0]:
#Procesamiento de fechas
for columna_fecha in fechas:
    final_df = final_df.withColumn(columna_fecha['columna'], F.regexp_replace(F.col(columna_fecha['columna']), r'\s', ''))
    final_df = final_df.withColumn(columna_fecha['columna'], F.to_date(F.col(columna_fecha['columna']), columna_fecha['formato']))

### Valores unicos por columna datos transformados

In [0]:
#Obtiene total registros
total_registros = final_df.count()
    
#Mostrar información por cada columna sin tener en cuenta IDs de cruce
for column in final_df.columns:
    displayHTML(f'<h3>{column}</h3>')
    display(final_df.select(column).groupBy(column).count().withColumn('porcentaje', F.round((F.col('count') / total_registros)*100.0, 2)))

ID_CONTACTO

ID_CONTACTO,count,porcentaje
0031I00000CMFbUQAX,1,0.0
0031I00000UvQI8QAN,1,0.0
0031I00000H2L4FQAV,1,0.0
0031I00000tX3OaQAK,1,0.0
0031I00000tX3tWQAS,1,0.0
0031I00000rJQbWQAW,1,0.0
0031I00000rJQC8QAO,1,0.0
0031I00000rJQCnQAO,1,0.0
0031I00000jTAAfQAO,1,0.0
0031I00000UuX3fQAF,1,0.0


ESTADO_CIVIL

ESTADO_CIVIL,count,porcentaje
null,187367,57.74
C - CASADO,38518,11.87
U - UNION LIBRE,1841,0.57
S - SOLTERO,95067,29.3
D - DIVORCIADO,766,0.24
P - SEPARADO,779,0.24
V - VIUDO,173,0.05


FECHA_NACIMIENTO

...

MONTO_TOTAL,count,porcentaje
135000,12,0.0
113000,2,0.0
4155000,1,0.0
5300,1,0.0
450000,70,0.02
870000,7,0.0
85000,21,0.01
3550000,2,0.0
1205000,1,0.0
234705,1,0.0


DONACIONES

DONACIONES,count,porcentaje
26,8,0.0
29,9,0.0
19,12,0.0
54,1,0.0
0,362,0.11
22,12,0.0
7,260,0.08
34,5,0.0
32,3,0.0
31,4,0.0


OPORTUNIDADES

OPORTUNIDADES,count,porcentaje
29,23,0.01
26,26,0.01
65,15,0.0
191,1,0.0
54,20,0.01
19,28,0.01
113,4,0.0
155,1,0.0
112,2,0.0
167,1,0.0


## 3. Guardado de informacion en silver

In [0]:
final_df.columns

Out[28]: ['ID_CONTACTO',
 'ESTADO_CIVIL',
 'FECHA_NACIMIENTO',
 'EDAD',
 'GENERACION',
 'GENERO',
 'PAIS',
 'CIUDAD',
 'DEPARTAMENTO',
 'DIRECCION',
 'ES_GRADUADO',
 'ES_EXALUMNO',
 'ES_ANTIGUO',
 'ES_ESTUDIANTE',
 'ES_MIEMBRO_CONCEJO',
 'ES_PADRE_MADRE',
 'ES_DONANTE',
 'ES_RESPONSABLE_FINANCIERO',
 'ES_ADMINISTRATIVO',
 'ES_PROFESOR',
 'ES_ANTIGUO_MIEMBRO_CONSEJO_SUPERIOR',
 'ES_DONANTE_LOS_ANDES_FOUNDATION',
 'ES_PROFESOR_ADMINISTRATIVO_RETIRADO',
 'VOLUNTARIO_UNIANDES',
 'ES_EGRESADO_CERTIFICADO',
 'ESTRATO',
 'PUNTAJE_RFM',
 'CARGO',
 'NIVEL_JERARQUICO',
 'SECTOR_ECONOMICO_1',
 'SECTOR_ECONOMICO_2',
 'EMPLEADOR',
 'NIT',
 'Empleador__c',
 'MONTO_TOTAL',
 'DONACIONES',
 'OPORTUNIDADES']

In [0]:
#Conversion a pandas
final_pd_df = final_df.toPandas()

In [0]:
final_pd_df.head()

Out[30]:

,ID_CONTACTO,ESTADO_CIVIL,FECHA_NACIMIENTO,EDAD,GENERACION,GENERO,PAIS,CIUDAD,DEPARTAMENTO,DIRECCION,ES_GRADUADO,ES_EXALUMNO,ES_ANTIGUO,ES_ESTUDIANTE,ES_MIEMBRO_CONCEJO,ES_PADRE_MADRE,ES_DONANTE,ES_RESPONSABLE_FINANCIERO,ES_ADMINISTRATIVO,ES_PROFESOR,ES_ANTIGUO_MIEMBRO_CONSEJO_SUPERIOR,ES_DONANTE_LOS_ANDES_FOUNDATION,ES_PROFESOR_ADMINISTRATIVO_RETIRADO,VOLUNTARIO_UNIANDES,ES_EGRESADO_CERTIFICADO,ESTRATO,PUNTAJE_RFM,CARGO,NIVEL_JERARQUICO,SECTOR_ECONOMICO_1,SECTOR_ECONOMICO_2,EMPLEADOR,NIT,Empleador__c,MONTO_TOTAL,DONACIONES,OPORTUNIDADES
0,0031I00000AibubQAB,S - SOLTERO,1986-03-01,36.0,Generación Y (Millennials),Masculino,COLOMBIA,BOGOTA D.C.,BOGOTA,CL 56 71 54,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,None,ASESOR FINANCIERO,Profesional,Actividades jurídicas y de contabilidad,"Actividades profesionales, científicas y técnicas",CARRIZOSA CONSULTORES SAS,9005276702,0011I00001PhB8jQAF,NaN,NaN,NaN
1,0031I00000AibweQAB,None,None,NaN,None,Femenino,COLOMBIA,BOGOTA D.C.,BOGOTA,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN
2,0031I00000AibwHQAR,None,None,NaN,None,Femenino,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN
3,0031I00000AibwkQAB,None,None,NaN,None,Masculino,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN
4,0031I00000AibwLQAR,S - SOLTERO,1998-09-30,24.0,Generación Z (Centennials),Femenino,COLOMBIA,CAJICA,CUNDINAMARCA,VRD 12 MCP CAJICA CA 304 CN PUERTA DEL SOL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,None,None,None,None,None,None,None,None,NaN,NaN,NaN


In [0]:
#Ruta de guardado
save_path = f'/dbfs{base_path}/silver/consolidacion_donaciones'

In [0]:
#Verifica si la ruta de guardado ya existe, en caso de que no la crea
import os
if not os.path.exists(f'{save_path}/'):
    os.makedirs(f'{save_path}/')

In [0]:
#Guardado de información en un único archivo
final_pd_df.to_parquet(f'{save_path}/consolidado_donaciones.parquet')